In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**import library**

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

**panggil data**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Semester5/STKI/ProjectUAS/SearchEngine/static/Databaru.csv')

In [5]:
data

,uri,artis_judulLagu,Label
0,0UATU9OJxh4m3fwDljdGZn,Evalyn - Creme de la creme,"wake up, get paid'noth dollar, 'nother daygo a..."
1,4JH1M62gVDNDhDAUiQB3Qv,Café Tacvba - La muerte chiquita,dame la muert chiquitadam la muert pequeñay as...
2,0h7Ld5CvgzaUN1zA3tdyPq,"Dawn Richard - we, diamonds",new breedstop call me ambiti when i dream fart...
3,4S1bYWrLOC8smuy8kJzxKQ,"Ricky Martin - Te Extraño, Te Olvido, Te Amo",traigo en lo bolsillo tanta soledaddesd que te...
4,758mQT4zzlvBhy9PvNePwC,A Tribe Called Quest - Butter,1988 senior year at garvey highwher all the gu...
...,...,...,...
8201,6qcvBgioxWYn1vPXJlKou3,Efek Rumah Kaca - Jalang,siapa yang berani bernyanyi nanti akan dikebir...
8202,36YxF0N7WM32JmtXwEBM39,The Adams - Jikalau,jikalau telah datang waktu yang dinanti kupast...
8203,55kXXCCGYokhnIB1hm5qYB,Goodnight Electric - Rocket Ship Goes By,i rememb when you said we have come from outsi...
8204,3XJDtP8dGjZ6RtNQ5AIWhv,Naif - Hidup Itu Indah,saat kau buka mata kita dilahirkan untuk sama ...


**menggabungkan semua kolom**

In [6]:
data['Search'] = data['artis_judulLagu'] + " " + data['Label']

In [7]:
data['Search'].head()

,Search
0,"Evalyn - Creme de la creme wake up, get paid'n..."
1,Café Tacvba - La muerte chiquita dame la muert...
2,"Dawn Richard - we, diamonds new breedstop call..."
3,"Ricky Martin - Te Extraño, Te Olvido, Te Amo t..."
4,A Tribe Called Quest - Butter 1988 senior year...


**modeling TF IDF**

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Langkah 1: Buat Model TF-IDF
tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.85)  # Menambahkan max_df untuk mengurangi kata yang terlalu sering muncul
vectors = tfidf.fit_transform(data['Search'])  # Vektor TF-IDF untuk seluruh data


**Terapkan LSA untuk reduksi dimensi**

In [9]:
from sklearn.decomposition import TruncatedSVD

# Langkah 2: Terapkan LSA untuk reduksi dimensi
lsa = TruncatedSVD(n_components=100, random_state=42)  # Menurunkan dimensi menjadi 100
lsa_vectors = lsa.fit_transform(vectors)


**Hitung Similarity Matrix**

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Langkah 3: Hitung Similarity Matrix
similarity = cosine_similarity(lsa_vectors)  # Matriks kemiripan antara semua entri


**buat function Search Engine**

In [15]:
def SearchEngine(query, top_n=5):
    # Transformasikan query menggunakan model TF-IDF
    query_vector = tfidf.transform([query])

    # Transformasikan query ke dalam ruang LSA
    query_lsa = lsa.transform(query_vector)

    # Hitung cosine similarity antara query LSA dan dataset LSA
    query_similarity = cosine_similarity(query_lsa, lsa_vectors)

    # Sortir hasil berdasarkan skor similarity
    sorted_similarities = sorted(
        list(enumerate(query_similarity[0])),
        reverse=True, key=lambda x: x[1]
    )

    # Gabungkan hasil pencarian model dan hasil pencarian langsung
    results = []
    # Pencarian langsung di kolom lirik
    filtered_data = data[data['Label'].str.contains(query, case=False, na=False)].head(top_n)
    for _, row in filtered_data.iterrows():
        song_info = {
            'artis_judulLagu': row['artis_judulLagu'],
            'snippet': row['Label']  # Potongan lirik dari dataset
        }
        results.append(song_info)
    # Pencarian berdasarkan model
    for i in sorted_similarities[:top_n]:
        index = i[0]
        similarity_score = i[1]
        song_info = {
            'artis_judulLagu': data.iloc[index]['artis_judulLagu'],
            'similarity': round(similarity_score * 100, 2)  # Skor similarity dalam persen
        }
        results.append(song_info)

    return results


**Buat Fungsi SearchEngine yang menggunakan LSA untuk pencarian**

In [16]:
from sklearn.decomposition import TruncatedSVD

# Terapkan LSA untuk reduksi dimensi
lsa = TruncatedSVD(n_components=100, random_state=42)  # Menurunkan dimensi menjadi 100
lsa_vectors = lsa.fit_transform(vectors)

# Fungsi SearchEngine yang menggunakan LSA untuk pencarian
def SearchEngine_LSA(query, top_n=5):
    # Transformasikan query menggunakan model TF-IDF
    query_vector = tfidf.transform([query])

    # Transformasikan query ke dalam ruang LSA
    query_lsa = lsa.transform(query_vector)

    # Hitung cosine similarity antara query LSA dan dataset LSA
    query_similarity = cosine_similarity(query_lsa, lsa_vectors)

    # Sortir hasil berdasarkan skor similarity
    sorted_similarities = sorted(
        list(enumerate(query_similarity[0])),
        reverse=True, key=lambda x: x[1]
    )

    # Ambil top_n hasil pencarian
    results = []
    for i in sorted_similarities[:top_n]:
        index = i[0]
        similarity_score = i[1]
        song_info = data.iloc[index]['search']
        results.append((song_info, similarity_score))

    return results


**testing model**

In [17]:
# Bagian 5: Query dan Hasil Pencarian
query = "jika hati ini bicara mungkinkah"  # Ganti dengan query pencarian Anda
results = SearchEngine(query)

# Hasil pencarian
print("\nHasil Pencarian:")
for result in results:
    if 'similarity' in result:
        print(f"Lagu: {result['artis_judulLagu']}, Similarity: {result['similarity']}%")
    if 'snippet' in result:
        print(f"Lagu: {result['artis_judulLagu']}, Lirik: {result['snippet']}")



Hasil Pencarian:
Lagu: Superglad - Suara Hati, Lirik: jika hati ini bicara mungkinkah terungkap semua rasa marah yang tersisa? dan jika hati ini terluka mungkinkah tersenyum walau asa ini tersiksa? untuk semua rasa cinta pernahkah kita sale mengalah? untuk semua rasa benci pernahkah kita sale memiliki? jika hati ini bicara mungkinkah terdengar semua rasa rindu yang ada? dan jika hati ini bertanya ... superglad indonesianindi indonesianpunk indonesianrock
Lagu: Superglad - Suara Hati, Similarity: 86.16%
Lagu: Repvblik Band - Tiada Guna Lagi, Similarity: 79.03%
Lagu: Kerispatih - Demi Cinta, Similarity: 77.2%
Lagu: Tony Q Rastafara - Kangen, Similarity: 76.37%
Lagu: Anarcute - Nikmat Patah Hati, Similarity: 76.34%


**Smpan ke pickle**

In [18]:
import pickle

# Simpan model ke file pickle
with open('tfidf_model.pkl', 'wb') as file:
    pickle.dump(tfidf, file)

with open('lsa_model.pkl', 'wb') as file:
    pickle.dump(lsa, file)

with open('lsa_vectors.pkl', 'wb') as file:
    pickle.dump(lsa_vectors, file)
